In [1]:
import re
import subprocess

import dxdata
import dxpy
import pandas as pd
import pyspark

from pyspark.sql import functions as F
from pyspark.conf import SparkConf
from pyspark.sql.types import StringType

from pathlib import Path
from phenotypes import *

Path("../tmp").resolve().mkdir(parents=True, exist_ok=True)

trait = "metabolic"

In [2]:
conf = SparkConf()
conf.set("autoBroadcastJoinThreshold", -1)

sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

dispensed_database_name = dxpy.find_one_data_object(
    classname="database", name="app*", folder="/", name_mode="glob", describe=True
)["describe"]["name"]
dispensed_dataset_id = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)["id"]

spark.sql("USE " + dispensed_database_name)

dataset = dxdata.load_dataset(id=dispensed_dataset_id)
participant = dataset["participant"]

In [3]:
first_occurence_fields = list(
    participant.find_fields(
        lambda f: bool(re.match("^Date (E10|E11|E66) first reported", f.title))
    )
)

age_sex_fields = get_age_sex(participant, fields = ["31", "21022"])

metabolic_fields = get_pheno_fields(participant, fields = [
        "130792",
        "2443",
        "130708",
        "130706",
    ]
)

field_names = concatenate(["eid"], age_sex_fields, first_occurence_fields, metabolic_fields)

df = participant.retrieve_fields(names=field_names, engine=dxdata.connect())

[dxdata.dataset.dataset] WARNING: Dataframe schema name(s): ['participant_0001$eid', 'participant_0001$p31', 'participant_0042$p21022', 'participant_0085$p130706', 'participant_0085$p130708', 'participant_0085$p130792', 'participant_0006$p2443_i0', 'participant_0006$p2443_i1', 'participant_0006$p2443_i2', 'participant_0006$p2443_i3'] not found in ordered columns: ['participant$eid', 'participant$p31', 'participant$p21022', 'participant$p130706', 'participant$p130708', 'participant$p130792', 'participant$p2443_i0', 'participant$p2443_i1', 'participant$p2443_i2', 'participant$p2443_i3']


In [5]:
df = fix_colnames(df)
df = filter_to_200k(df)

In [23]:
df.write.csv("/tmp/phenos.tsv", sep="\t", header=True, emptyValue='NA')

In [24]:
subprocess.run(
    ["hadoop", "fs", "-rm", "/tmp/phenos.tsv/_SUCCESS"], check=True, shell=False
)
subprocess.run(
    ["hadoop", "fs", "-get", "/tmp/phenos.tsv", "../tmp/phenos.tsv"],
    check=True,
    shell=False,
)

CompletedProcess(args=['hadoop', 'fs', '-get', '/tmp/phenos.tsv', '../tmp/phenos.tsv'], returncode=0)

In [25]:
!sed -e '3,${/^xeid/d' -e '}' ../tmp/phenos.tsv/part* > ../tmp/metabolic.BT.raw.tsv

In [26]:
# Upload to project

subprocess.run(
    ["dx", "upload", "../tmp/metabolic.BT.raw.tsv", "--path", "Data/phenotypes/"],
    check=True,
    shell=False,
)

CompletedProcess(args=['dx', 'upload', '../tmp/metabolic.BT.raw.tsv', '--path', 'Data/phenotypes/'], returncode=0)